# Einführung in die Parallelisierung

## Martelli’s Modell der Skalierbarkeit

| Anzahl Kerne | Beschreibung                           |
| ------------ | -------------------------------------- |
| 1            | Einzelner Thread und einzelner Prozess |
| 2–8          | Mehrere Threads und mehrere Prozesse   |
| ≥9           | Verteilte Verarbeitung                 |

Martelli’s Beobachtung: Im Laufe der Zeit wird die zweite Kategorie immer unbedeutender: Einzelne Kerne werden immer leistungsfähiger und große Datensätze immer größer.

## Global Interpreter Lock (GIL)

CPython verfügt über eine Sperre für seinen intern geteilten globalen Status. Dies hat zur Folge, dass nicht mehr als ein Thread gleichzeitig laufen kann.

Für I/O-lastige Anwendungen ist das GIL kein großes Problem; bei CPU-lastigen Anwendungen führt die Verwendung von Threading jedoch zu einer Verlangsamung. Dementsprechend ist Multi-Processing für uns spannend um mehr CPU-Zyklen zu erhalten.

## Threads vs. Prozesse vs. Async

### Threads

#### Pros

* Threads haben den Vorteil, einen gemeinsamen Status zu teilen. Allerdings kann dies auch zu *Race Conditions* führen, d.h., die Ergebnisse einer Operation können vom zeitlichen Verhalten bestimmter Einzeloperationen abhängen.

* Threads wechseln präemptiv, s. [Präemptives Multitasking](https://de.wikipedia.org/wiki/Multitasking#Pr%C3%A4emptives_Multitasking). Dies ist praktisch, da Ihr keinen expliziten Code hinzufügen müsst, um einen Wechsel der Tasks zu verursachen.

* Threading funktioniert normalerweise mit vorhandenem Code und Werkzeugen, solange Locks für kritische Abschnitte hinzugefügt werden.

* Threads erfordern sehr wenig Tooling: [Lock](https://docs.python.org/3/library/threading.html#threading.Lock) und [Queues](https://docs.python.org/3/library/queue.html).

#### Cons

* Die Kosten für diese Bequemlichkeit sind, dass ihr davon ausgehen müsst, dass ein solcher Wechsel jederzeit möglich ist. Dementsprechend müssen kritische Bereiche mit Locks geschützt werden.

* Die Leistungsgrenze für Threads ist eine CPU abzüglich der Kosten für Task-Switches und Synchronisationsaufwände.

### Prozesse

#### Pros

* Die Stärke von Prozessen ist, unabhängig voneinander zu sein.

#### Cons

* Allerdings kommunizieren sie auch nicht miteinander. Daher werden [Interprocess Communication (IPC)](https://docs.python.org/3/library/ipc.html), [object pickling](https://docs.python.org/3/library/pickle.html) und anderer Overhead notwendig.

### Async

#### Pros

* Async schaltet kooperativ um, daher müsst ihr explizit [yield](https://docs.python.org/3/reference/simple_stmts.html#yield) oder [await](https://docs.python.org/3/reference/expressions.html#await) hinzufügen, um einen Task-Switch herbeizuführen. Damit könnt ihr kontrollieren, wann diese Task-Switches und ggf. Locks und Synchronisationen stattfinden sollen. Ihr könnt daher die Aufwände für Task-Switches sehr niedrig halten. Zudem hat der Aufruf einer reinen Python-Funktion mehr Overhead als die erneute Anfrage eines *generator* oder *awaitable* – d.h., Async ist sehr billig.

* Async kann die CPU-Auslastung verbessern, da es die üblichen Aufwände reduzieren kann.
* Bei komplexen Systemen kommt man mit async viel einfacher zum Ziel als mit Threads mit Locks.

#### Cons

* Async benötigt eine große Menge an Werkzeugen: [futures](https://docs.python.org/3/library/asyncio-task.html#future), [Event Loops](https://docs.python.org/3/library/asyncio-eventloops.html) und nicht-blockierende Versionen von fast allem.